In [1]:
# Required Packages
import pandas as pd
import numpy as np

In [2]:
# Load Data
trans_3w = pd.read_csv(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\trans_3w.csv')
trans_3w.head()

,txn_hash,input_address,output_address,ammount,fees,block_index,block_time,input_flag,output_flag
0,bd36f2ca16e2a2c73c807b7d1569657b30de8453450cd2...,13Uf71d8y94xEk2LX7GCtaBJmPiahhA7TR,16FPyvvz5Ug3cx97qH67KfgC6PY1S9fskQ,2.420000e+07,320000.000000,453318,2017-02-16 12:05:04,0,1
1,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,166zajP74bcRVo7BmdeDME3mRX3Mi9e3xn,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,2.503648e+03,3.314845,453318,2017-02-16 12:05:04,0,1
2,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,6.359402e+05,841.988605,453318,2017-02-16 12:05:04,0,1
3,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,1HVQNFf7vDpJVZk7tEzbFxnmALSezA2qPD,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,5.902369e+05,781.477250,453318,2017-02-16 12:05:04,0,1
4,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,1.013032e+05,134.126076,453318,2017-02-16 12:05:04,0,1


In [3]:
# Get Subset of Transactions with Negative Ammounts
neg = trans_3w[trans_3w['ammount'] < 0]
neg.shape

(70756, 9)

In [4]:
# Download Data
#neg.to_csv("neg_ammount.csv", index = False)

Intuitively, the presence of negative ammounts does not make sense. In speaking to the professors of the blockchain class we came to the conclusion that this is an anomaly which should not be present. The DataFrame was sent to Lluc to assess if this anomaly was a result of their pre-processing.

*Check for Channge when Self-Loops are Removed*

In [5]:
trans_3w_adj = pd.read_csv(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\trans_3w_cl.csv')
neg2 = trans_3w_adj[trans_3w_adj['ammount'] < 0]
neg2.shape # Shows some reduction

(49788, 9)

*Check Properties of these addresses in the Network*

In [7]:
neg_ad = set(neg2['input_address'].tolist() + neg2['output_address'].tolist())
print('Number of unique addresses with negative transaction ammounts : ', len(neg_ad))

Number of unique addresses with negative transaction ammounts :  53212


In [8]:
# Get Node properties of listed addresses
properties_cl = pd.read_csv(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\node_properties_cl.csv')
node_prop = properties_cl[properties_cl['address'].isin(neg_ad)]
node_prop.describe()

,degree,in_degree,out_degree,eigen_centrality,fraud_flag
count,53212.000000,53212.000000,53212.000000,5.321200e+04,53212.000000
mean,14.492765,7.079399,7.413365,3.094047e-05,0.002293
std,202.078341,107.165015,154.606607,6.564628e-04,0.047828
min,1.000000,0.000000,0.000000,1.193322e-21,0.000000
25%,2.000000,1.000000,1.000000,1.312654e-20,0.000000
50%,3.000000,1.000000,2.000000,6.682603e-20,0.000000
75%,3.000000,1.000000,2.000000,1.221962e-18,0.000000
max,25338.000000,12116.000000,25335.000000,1.049846e-01,1.000000


In [11]:
n_degree = node_prop[node_prop['degree'] > 14.5]
n_degree.shape

(4370, 6)

1. These nodes have a large degree on average. Mean is potentially biased from the very long positive tail of the distribution. This is also indicated by the large standard deviation.
2. The distribution of centralities is similar to that of the overall network.
3. Addresses are largely involved in licit transactions.

In [13]:
neg_nodes1 = trans_3w_adj[trans_3w_adj['input_address'].isin(neg_ad)]
neg_nodes2 = trans_3w_adj[trans_3w_adj['output_address'].isin(neg_ad)]
neg_nodes = pd.concat([neg_nodes1, neg_nodes2])
neg_nodes.shape

(1396637, 9)

This shows that the addresses are involved in multiple transactions such that removing the individual transactions with negative ammounts would not significantly impact the network, but the addresses involved in these transactions should not be removed.